In [ ]:
!nvidia-smi

Tue Jan  5 11:13:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import numpy as np
import os, sys
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Colab_NLP/tianchi-multi-task-nlp"

nb_path = '/content/drive/MyDrive/Colab_NLP/tianchi-multi-task-nlp/'
sys.path.insert(0,nb_path)

In [ ]:
pip install keras-bert #keras tensorflow pycaret fasttext

  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=a5ed7e28ffe7763e3fb9f7c4a46b2849d1b16504d4514d06f399fb145fc3d4db
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for keras-transformer: filename=keras_transformer-0.38.0-cp36-none-any.whl size=12944 sha256=ac019c63edadd9c31247a11a9131f327850465baa24f586d2352d32671031d53
  Stored in directory: /root/.cache/pip/wheels/e5/fb/3a/37b2b9326c799aa010ae46a04ddb04f320d8c77c0b7e837f4e
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7553 sha256=2ae0729243d120f655c4b632b340b985835fc9a713649f054da1a972e7e52ed9
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp36-none-any.whl size=15612 sha256=5d9a916b0e28d86db6879a3179a012c1e4d1dacab07b6f0d52b767eadcc8233c
  Stor

In [ ]:
import pandas as pd
import codecs, gc
import numpy as np
from sklearn.model_selection import KFold
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
from keras.metrics import top_k_categorical_accuracy
from keras.layers import *
from keras.callbacks import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [ ]:
#将ocnli中content1+content2作为content输入
#times_train = pd.read_csv(path + '/tianchi_datasets/TNEWS/total.csv',  sep='\t', header=None, names=('id', 'content', 'label')).astype(str)
ocemo_train = pd.read_csv(path + '/tianchi_datasets/OCEMOTION/total.csv',sep='\t', header=None, names=('id', 'content', 'label')).astype(str)
# ocnli_train = pd.read_csv(path + '/tianchi_datasets/OCNLI/total.csv',  sep='\t', header=None, names=('id', 'content1', 'content2', 'label')).astype(str)
# ocnli_train['content'] = ocnli_train['content1'] + ocnli_train['content2']
#/content/drive/MyDrive/Colab_NLP/tianchi-multi-task-nlp/TEST/ocemotion_test_B.csv
#times_testa = pd.read_csv(path + '/tianchi_datasets/TNEWS/test.csv',  sep='\t', header=None, names=('id', 'content')).astype(str)
ocemo_testa = pd.read_csv(path + '/TEST/ocemotion_test_B.csv',sep='\t', header=None, names=('id', 'content')).astype(str)
# ocnli_testa = pd.read_csv(path + '/tianchi_datasets/OCNLI/test.csv',  sep='\t', header=None, names=('id', 'content1', 'content2')).astype(str)
# ocnli_testa['content'] = ocnli_testa['content1'] + ocnli_testa['content2']

In [ ]:
#构造输入、标签数据
train_df = ocemo_train#pd.concat([times_train, ocemo_train, ocnli_train[['id','content', 'label']]], axis=0).copy()
testa_df = ocemo_testa#pd.concat([times_testa, ocemo_testa, ocnli_testa[['id', 'content']]], axis=0).copy()
#label需要从0开始
encode_label = LabelEncoder()
train_df['label'] = encode_label.fit_transform(train_df['label'].apply(str))

#train_df[['label']].to_csv(path + "/subs/trnlabel.csv", index=None)

###
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)

X_trn = pd.DataFrame()
X_val = pd.DataFrame()

for train_index, test_index in skf.split(train_df.copy(), train_df['label']):
  X_trn, X_val = train_df.iloc[train_index], train_df.iloc[test_index]
  break
###

subs = testa_df[['id']]
testa_df = testa_df[['content']]

#训练数据、测试数据和标签转化为模型输入格式
n_cls = len( train_df['label'].unique() )
TRN_LIST = []
for data_row in X_trn.iloc[:].itertuples():
  TRN_LIST.append((data_row.content, to_categorical(data_row.label, n_cls)))
TRN_LIST = np.array(TRN_LIST)

VAL_LIST = []
for data_row in X_val.iloc[:].itertuples():
  VAL_LIST.append((data_row.content, to_categorical(data_row.label, n_cls)))
VAL_LIST = np.array(VAL_LIST)
 
DATA_LIST_TEST = []
for data_row in testa_df.iloc[:].itertuples():
  DATA_LIST_TEST.append((data_row.content, to_categorical(0, n_cls)))
DATA_LIST_TEST = np.array(DATA_LIST_TEST)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
print(train_df.shape, testa_df.shape)

(35315, 3) (1500, 1)


In [ ]:
!unzip /content/drive/MyDrive/Colab_NLP/chinese_roberta_wwm_large_ext_L-24_H-1024_A-16.zip 

Archive:  /content/drive/MyDrive/Colab_NLP/chinese_roberta_wwm_large_ext_L-24_H-1024_A-16.zip
  inflating: bert_config.json        
  inflating: bert_model.ckpt.data-00000-of-00001  
  inflating: bert_model.ckpt.index   
  inflating: bert_model.ckpt.meta    
  inflating: vocab.txt               


In [ ]:
#预训练好的模型
path_bert = '/content/chinese_roberta_wwm_large_ext_L-24_H-1024_A-16/'
config_path = '/content/bert_config.json'
dict_path  = '/content/vocab.txt'
checkpoint_path ='/content/bert_model.ckpt'

#一些自定义参数
er_patience = 2 #early_stopping patience
lr_patience = 2 #ReduceLROnPlateau patience
max_epochs = 3 #epochs
lr_rate = 2e-6 #learning rate
batch_sz = 6 #batch_size
maxlen = 158  #设置序列长度为，要保证序列长度不超过512 ??
nfold = 5
'''
#0.58分数
er_patience = 2 #early_stopping patience
lr_patience = 3 #ReduceLROnPlateau patience
max_epochs = 3 #epochs
lr_rate = 1e-5 #learning rate
batch_sz = 16 #batch_size
maxlen = 64  #设置序列长度为，要保证序列长度不超过512 ??
'''
#将词表中的词编号转换为字典
token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
  for line in reader:
    token = line.strip()
    token_dict[token] = len(token_dict)

#重写tokenizer        
class OurTokenizer(Tokenizer):
  def _tokenize(self, text):
    R = []
    for c in text:
      if c in self._token_dict:
        R.append(c)
      elif self._is_space(c):
        R.append('[unused1]')  # 用[unused1]来表示空格类字符
      else:
        R.append('[UNK]')  # 不在列表的字符用[UNK]表示
    return R

tokenizer = OurTokenizer(token_dict)
 
#让每条文本的长度相同，用0填充
def seq_padding(X, padding=0):
  L = [len(x) for x in X]
  ML = max(L)
  return np.array([  np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X  ])
  
#data_generator只是一种为了节约内存的数据方式
class data_generator:
  global batch_sz
  def __init__(self, data, batch_size=batch_sz, shuffle=True):#此处修改batch_size
    self.data = data
    self.batch_size = batch_size
    self.shuffle = shuffle
    self.steps = len(self.data) // self.batch_size
    if len(self.data) % self.batch_size != 0:
      self.steps += 1

  def __len__(self):
    return self.steps

  def __iter__(self):
    while True:
      idxs = list(range(len(self.data)))
      if self.shuffle:
        np.random.shuffle(idxs)
      X1, X2, Y = [], [], []
      for i in idxs:
        d = self.data[i]
        text = d[0][:maxlen]
        x1, x2 = tokenizer.encode(first=text)
        y = d[1]
        X1.append(x1)
        X2.append(x2)
        Y.append([y])
        if len(X1) == self.batch_size or i == idxs[-1]:
          X1 = seq_padding(X1)
          X2 = seq_padding(X2)
          Y = seq_padding(Y)
          yield [X1, X2], Y[:, 0, :]
          [X1, X2, Y] = [], [], []

#计算top-k正确率,当预测值的前k个值中存在目标类别即认为预测正确                 
def acc_top2(y_true, y_pred):
  return top_k_categorical_accuracy(y_true, y_pred, k=2)

def boolMap(arr):
  if arr > 0.5:
      return 1
  else:
      return 0

#自定义计算每个epoch的F1，注意不是batch的F1
class Metrics(Callback):
  def __init__(self, filepath):
    self.file_path = filepath

  def on_train_begin(self, logs=None):
    self.val_f1s = []
    self.best_val_f1 = 0
    self.val_recalls = []
    self.val_precisions = []

  def on_epoch_end(self, epoch, logs=None):
    val_predict = list(map(boolMap, self.model.predict([self.validation_data[0], self.validation_data[1]])))
    val_targ = self.validation_data[2]
    _val_f1 = f1_score(val_targ, val_predict)
    _val_recall = recall_score(val_targ, val_predict)
    _val_precision = precision_score(val_targ, val_predict)
    self.val_f1s.append(_val_f1)
    self.val_recalls.append(_val_recall)
    self.val_precisions.append(_val_precision)
    print(_val_f1, _val_precision, _val_recall)
    print("max f1")
    print(max(self.val_f1s))
    if _val_f1 > self.best_val_f1:
      self.model.save_weights(self.file_path, overwrite=True)
      self.best_val_f1 = _val_f1
      print("best f1: {}".format(self.best_val_f1))
    else:
      print("val f1: {}, but not the best f1".format(_val_f1))
    return

f1metrics = Metrics(path)

#计算每个batch的f1
from keras import backend as K

def f1(y_true, y_pred):
  def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

  def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
  precision = precision(y_true, y_pred)
  recall = recall(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

#bert模型设置
def build_bert(nclass):
  global lr_rate
  bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)  #加载预训练模型

  for l in bert_model.layers:
    l.trainable = True

  x1_in = Input(shape=(None,))
  x2_in = Input(shape=(None,))

  x = bert_model([x1_in, x2_in])
  x = Lambda(lambda x: x[:, 0])(x) # 取出[CLS]对应的向量用来做分类
  p = Dense(nclass, activation='softmax')(x)

  model = Model([x1_in, x2_in], p)
  model.compile(loss='categorical_crossentropy',
                optimizer=Adam(lr_rate),    #用足够小的学习率
                metrics=['accuracy', f1])#acc_top2

  print(model.summary())
  return model

#交叉验证训练和测试模型
def run_nocv(nfold, trn_data, val_data, data_labels, data_test, n_cls):
  global er_patience
  global lr_patience
  global max_epochs
  global f1metrics
  test_model_pred = np.zeros((len(data_test), n_cls))
  '''    
  # 按照9:1的比例划分训练集和验证集
  random_order = list( range(len(data)) )
  np.random.shuffle(random_order)
  train_data = [data[j] for i, j in enumerate(random_order) if i % 10 != 0]
  valid_data = [data[j] for i, j in enumerate(random_order) if i % 10 == 0]
  '''

  model = build_bert(n_cls)
  early_stopping = EarlyStopping(monitor= "val_accuracy", patience=er_patience) #早停法，防止过拟合#'val_accuracy'
  plateau = ReduceLROnPlateau(monitor="val_accuracy", verbose=1, mode='max', factor=0.5, patience=lr_patience) #当评价指标不在提升时，减少学习率#"val_accuracy"
  checkpoint = ModelCheckpoint(path + "/subs/ocemo.epoch{epoch:02d}_val_loss{val_loss:.4f}_val_acc{val_accuracy:.4f}_val_f1{val_f1:.4f}.hdf5", monitor="val_accuracy", verbose=2, save_best_only=True, mode='max', save_weights_only=True) #保存最好的模型#'val_acc'

  train_D = data_generator(trn_data, shuffle=True)
  valid_D = data_generator(val_data, shuffle=True)
  test_D  = data_generator(data_test, shuffle=False)
  
  #模型训练
  model.fit_generator(
      train_D.__iter__(),
      steps_per_epoch=len(train_D),
      epochs=max_epochs,
      validation_data=valid_D.__iter__(),
      validation_steps=len(valid_D),
      callbacks=[early_stopping, plateau, checkpoint],
  )

  test_model_pred = model.predict_generator(test_D.__iter__(), steps=len(test_D), verbose=1)
  train_model_pred = model.predict(train_D.__iter__(), steps=len(train_D), verbose=1)
 
  gc.collect()   #清理内存
  del model
  K.clear_session() #clear_session就是清除一个session

  return test_model_pred, train_model_pred

In [ ]:
cvs = 5
test_model_pred, train_model_pred = run_nocv(cvs, TRN_LIST, VAL_LIST, None, DATA_LIST_TEST, n_cls)

preds_col_names = encode_label.inverse_transform( range(0,n_cls) )
preds_tst_df = pd.DataFrame(test_model_pred)
preds_trn_df = pd.DataFrame(train_model_pred)

preds_tst_df.columns = preds_col_names
preds_trn_df.columns = preds_col_names


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
model_1 (Functional)            (None, None, 1024)   324472832   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1024)         0           model_1[0][0]              

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


4709/4709 [==============================] - 1409s 291ms/step - loss: 1.3455 - accuracy: 0.5059 - f1: 0.3868 - val_loss: 1.0977 - val_accuracy: 0.6063 - val_f1: 0.5655

Epoch 00001: val_accuracy improved from -inf to 0.60626, saving model to /content/drive/MyDrive/Colab_NLP/tianchi-multi-task-nlp/subs/ocemo.epoch01_val_loss1.0977_val_acc0.6063_val_f10.5655.hdf5
Epoch 2/3
4709/4709 [==============================] - 1354s 287ms/step - loss: 0.9977 - accuracy: 0.6417 - f1: 0.6004 - val_loss: 1.0856 - val_accuracy: 0.6183 - val_f1: 0.5859

Epoch 00002: val_accuracy improved from 0.60626 to 0.61829, saving model to /content/drive/MyDrive/Colab_NLP/tianchi-multi-task-nlp/subs/ocemo.epoch02_val_loss1.0856_val_acc0.6183_val_f10.5859.hdf5
Epoch 3/3
4709/4709 [==============================] - 1351s 287ms/step - loss: 0.8115 - accuracy: 0.7073 - f1: 0.6827 - val_loss: 1.1320 - val_accuracy: 0.6118 - val_f1: 0.5954

Epoch 00003: val_accuracy did not improve from 0.61829


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


4709/4709 [==============================] - 280s 59ms/step


In [ ]:
preds_tst_df.head(ocemo_testa.shape[0])

,anger,disgust,fear,happiness,like,sadness,surprise
0,0.006839,0.049984,0.067279,0.012439,0.005056,0.857248,0.001156
1,0.000091,0.000046,0.000010,0.000045,0.000113,0.999689,0.000006
2,0.114630,0.036727,0.007626,0.063425,0.478429,0.163667,0.135496
3,0.001815,0.001175,0.002376,0.121482,0.361577,0.510859,0.000716
4,0.003547,0.003791,0.000868,0.960563,0.021864,0.005930,0.003437
...,...,...,...,...,...,...,...
1495,0.000271,0.000578,0.000085,0.001190,0.994515,0.002443,0.000918
1496,0.015257,0.243143,0.004671,0.672554,0.010390,0.050677,0.003309
1497,0.333023,0.120519,0.001163,0.004108,0.016267,0.518600,0.006321
1498,0.075615,0.053657,0.004608,0.038493,0.147233,0.651833,0.028560


In [ ]:
#times_testa, ocemo_testa, ocnli_testa
#每行，从每个任务对应的标签列中找出最大的概率对应的列名作为预测结果
ocemo_preds = preds_tst_df.head(ocemo_testa.shape[0])
ocemo_preds = ocemo_preds.eq(ocemo_preds.max(1), axis=0).dot(ocemo_preds.columns)

ocemo_sub = ocemo_testa[['id']].copy()
ocemo_sub['label'] = ocemo_preds.values
ocemo_sub.to_json(path + "/submission/sub/ocemo_predict.json", orient='records', lines=True)

In [ ]:
%cd /content/drive/MyDrive/Colab_NLP/tianchi-multi-task-nlp/subs/
!ls -l

/content/drive/MyDrive/Colab_NLP/tianchi-multi-task-nlp/subs
total 8874986
-rw------- 1 root root 1298195560 Jan  5 09:18 ocemo.epoch01_val_loss1.0977_val_acc0.6063_val_f10.5655.hdf5
-rw------- 1 root root 1298195560 Jan  5 09:41 ocemo.epoch02_val_loss1.0856_val_acc0.6183_val_f10.5859.hdf5
-rw------- 1 root root      46824 Jan  5 10:13 ocemo_predict_merge.json
-rw------- 1 root root 1298179176 Jan  5 10:52 ocnli.epoch01_val_loss0.5746_val_acc0.7610_val_f10.7537.hdf5
-rw------- 1 root root      37889 Dec 28 11:04 ocnli_predict.json
-rw------- 1 root root     290662 Dec 28 09:13 prob_sub1.csv
-rw------- 1 root root 1298228328 Dec 29 11:14 times.epoch02_val_loss1.1762_val_acc0.5795_val_f10.5606.hdf5
-rw------- 1 root root 1298228328 Dec 29 09:31 times.epoch02_val_loss1.1852_val_acc0.5777_val_f10.5633.hdf5
-rw------- 1 root root 1298228328 Dec 29 11:31 times.epoch03_val_loss1.1817_val_acc0.5833_val_f10.5755.hdf5
-rw------- 1 root root 1298228328 Dec 24 05:45 times.epoch03_val_loss1.4469_va

In [ ]:
# rebuild model as model2
# simulating starting a new script
model2 = build_bert(7)

# load 3rd epoch weights
#model2.load_weights('times.epoch03_val_loss1.4469_val_acc0.5833_val_f10.5835.hdf5')
model2.load_weights('ocemo.epoch02_val_loss1.0856_val_acc0.6183_val_f10.5859.hdf5')
test_D  = data_generator(DATA_LIST_TEST, shuffle=False)
test_model_pred = model2.predict_generator(test_D.__iter__(), steps=len(test_D), verbose=1)

preds_col_names = encode_label.inverse_transform( range(0,n_cls) )
preds_tst_df = pd.DataFrame(test_model_pred)

preds_tst_df.columns = preds_col_names

preds_tst_df.to_csv('/content/drive/MyDrive/Colab_NLP/tianchi-multi-task-nlp/submission/sub/prob_ocemo_sub_final.csv', index=None)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
model_1 (Functional)            (None, None, 1024)   324472832   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1024)         0           model_1[0][0]              

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


250/250 [==============================] - 36s 102ms/step


In [ ]:
%cd ~
%cd /content/drive/MyDrive/Colab_NLP/tianchi-multi-task-nlp/submission/sub
!ls -l

/root
/content/drive/MyDrive/Colab_NLP/tianchi-multi-task-nlp/submission/sub
total 1348
-rw------- 1 root root  46961 Jan  5 10:09 ocemo_predict.json
-rw------- 1 root root  37889 Dec 28 11:07 ocnli_predict.json
-rw------- 1 root root 122928 Jan  5 10:13 prob_ocemo_sub.csv
-rw------- 1 root root 121289 Jan  5 11:17 prob_ocemo_sub_final.csv
-rw------- 1 root root  53593 Dec 28 10:53 prob_ocnli_sub.csv
-rw------- 1 root root 290662 Dec 27 09:47 prob_sub1.csv
-rw------- 1 root root 291309 Jan  5 11:10 prob_sub.csv
-rw------- 1 root root 290670 Dec 29 11:35 prob_sub_tnews_1229.csv
-rw------- 1 root root  40889 Dec 29 11:35 tnews_predict_1229.json
-rw------- 1 root root  40889 Dec 24 07:55 tnews_predict.json
-rw------- 1 root root  40889 Dec 24 06:47 tnews_predict_sg.json


In [ ]:
# path = '/content/drive/MyDrive/Colab_NLP/tianchi-multi-task-nlp/submission'
preds_tst_df = pd.read_csv("prob_ocemo_sub_final.csv") #+ pd.read_csv( "/subs/prob_sub.csv")


#每行，从每个任务对应的标签列中找出最大的概率对应的列名作为预测结果
ocemo_preds = preds_tst_df.head(ocemo_testa.shape[0])
ocemo_preds = ocemo_preds.eq(ocemo_preds.max(1), axis=0).dot(ocemo_preds.columns)

ocemo_sub = ocemo_testa[['id']].copy()
ocemo_sub['label'] = ocemo_preds.values
ocemo_sub.to_json(path + "/subs/ocemo_predict_final.json", orient='records', lines=True)

# ocemo_sub = ocemo_testa[['id']].copy()
# ocemo_sub['label'] = ocemo_preds.values
# ocemo_sub.to_json(path + "/subs/ocemotion_predict.json", orient='records', lines=True)

# ocnli_sub = ocnli_testa[['id']].copy()
# ocnli_sub['label'] = ocnli_preds.values
# ocnli_sub.to_json(path + "/subs/ocnli_predict.json", orient='records', lines=True)